In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors, Word2Vec
from matplotlib import pyplot as plt
import gensim
import re

In [2]:
df = pd.read_csv('data/tokenized/in_domain_train.tsv', sep='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

In [3]:
df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,NaN,one more pseudo generalization and i 'm giving...
2,gj04,1,NaN,one more pseudo generalization or i 'm giving ...
3,gj04,1,NaN,"the more we study verbs , the crazier they get ."
4,gj04,1,NaN,day by day the facts are getting murkier .


In [4]:
df['sentence'] = df['sentence'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+",' ', x))
df['tokens'] = df['sentence'].apply(lambda x: x.strip().split(' '))

# Feature Engineering

In [5]:
w2vec = Word2Vec(
     min_count=20,
     window=2,
     size=300,
     sample=6e-5, 
     alpha=0.03, 
     min_alpha=0.0007, 
     negative=20,
     workers=8-1
)

In [6]:
sentences = df['sentence'].apply(lambda x: x.split(' ')).values
w2vec.build_vocab(sentences, progress_per=10000)

In [7]:
w2vec.train(sentences, total_examples=w2vec.corpus_count, epochs=30, report_delay=1)

(256861, 2258880)

In [8]:
w2vec.init_sims(replace=True)

In [17]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        
        if not mean:
            print("cannot compute similarity with no input %s", words)
            return np.zeros(wv.vector_size, )
        
        mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
        return mean
    
def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list])

In [11]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2020)

In [19]:
model = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
word_averaging_list(model, train_df['tokens'])

/home/superceed1/anaconda3/envs/scrapper/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/home/superceed1/anaconda3/envs/scrapper/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/superceed1/anaconda3/envs/scrapper/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


TypeError: 'NoneType' object is not subscriptable